In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2023-10-09_data.csv")

/var/folders/2v/zjk16qr55dv34_t6b41l4rph0000gn/T/ipykernel_17671/3383318221.py:1: DtypeWarning: Columns (89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("2023-10-09_data.csv")


In [3]:
# Define the list of columns to drop
drop_cols = ['Posudek znalce', 'Velikost podílu', 'Minimální kupní cena', 'Plocha bazénu', 'Termín 2. prohlídky', 'Dražební vyhláška', 'Termín 1. prohlídky', 'Plocha zahrady', 'Datum konání dražby', 'Datum prohlídky do', 'Vyvolávací cena', 'Znalecký posudek', 'Minimální příhoz', 'Dražební jistota', 'Druh dražby', 'Místo konání dražby', 'Datum prohlídky', 'Anuita', 'Stav', 'Počet bytů', 'Výška stropu', 'Cena', 'Datum ukončení výstavby', 'Datum zahájení prodeje', 'Zlevněno', 'Původní cena', 'Plocha zastavěná', 'Plocha podlahová', 'Převod do OV', 'Ukazatel energetické náročnosti budovy', 'Průkaz energetické náročnosti budovy', 'Doprava', 'Komunikace' , 'Voda', 'Topení', 'Odpad', 'Telekomunikace', 'Elektřina', 'Datum nastěhování', 'Rok kolaudace', 'Rok rekonstrukce', 'ID zakázky', 'Náklady na bydlení', 'ID', 'Aktualizace', 'Poznámka k ceně', 'Velikost podílu společných prostor']
# odebrat automaticky podle počtu
# Define the list of columns to one-hot encode
one_hot_encode_cols = ['Typ bytu', 'Umístění objektu', 'Vybavení', 'Vlastnictví', 'Stav objektu', 'Stavba', 'Energetická náročnost budovy']

# Define the list of binary columns
binary_cols = ['Bazén', 'Půdní vestavba', 'Plyn', 'Bezbariérový', 'Terasa', 'Garáž', 'Výtah', 'Sklep', 'Parkování', 'Balkón']
# pridat nazvy 
# Define the list of distance columns
distance_cols = ['Večerka', 'Cukrárna', 'Divadlo', 'Veterinář',  'Hospoda', 'Kino', 'Hřiště', 'Přírodní zajímavost', 'Kulturní památka', 'Sportoviště', 'Bankomat', 'Školka', 'Pošta', 'Vlak', 'Bus MHD', 'Škola', 'Restaurace', 'Lékař', 'Tram', 'Obchod', 'Lékárna', 'Metro']


In [4]:
# Drop the unnecessary columns
df = df.drop(columns=drop_cols)

# Clean 'url_id'
df['url_id'] = df['url_id'].str.replace('/cs/v2/estates/', '')

# Prepare 'Podlaží'
df['Podlaží'] = df['Podlaží'].str.extract('(\d+)', expand=False)
df['Podlaží'] = pd.to_numeric(df['Podlaží'], errors='coerce')

# Convert 'Celková cena' to numerical by removing whitespace and non-digit characters
df['Celková cena'] = df['Celková cena'].replace('\D', '', regex=True).astype(float)

# Correctly prepare 'Energetická náročnost budovy' by extracting the letter after "Třída"
df['Energetická náročnost budovy'] = df['Energetická náročnost budovy'].str.extract('Třída (\w)', expand=False)

# One-hot encode the specified columns
df = pd.get_dummies(df, columns=one_hot_encode_cols, drop_first=True)

# Convert the binary columns to 1 or 0
for col in binary_cols:
    df[col] = df[col].notnull().astype(int)

# Drop rows where both 'url_id' and 'Celková cena' are missing
df = df.dropna(subset=['url_id', 'Celková cena'], how='all')

# Convert any non-missing value in 'Lodžie' to 1 and missing values to 0
df['Lodžie'] = df['Lodžie'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Drop rows where either 'url_id' or 'Celková cena' are missing
df = df.dropna(subset=['url_id', 'Celková cena'], how='any')

# Fill missing values in 'Podlaží' with 0
df.loc[:, 'Podlaží'] = df['Podlaží'].fillna(0)

In [5]:
# Fill missing values in distance columns with 10000 (representing 10 kilometers)
df[distance_cols] = df[distance_cols].fillna(10000)

def process_and_encode_distance(df, one_hot_encode_distance=False):
    # Define the bins and labels
    bins = [0, 500, 1000, 2000, 5000, float('inf')]
    labels = ['Very Close', 'Close', 'Medium Distance', 'Far', 'Very Far']

    # Apply binning to the distance columns
    for col in distance_cols:
        df[col] = pd.cut(df[col], bins=bins, labels=labels, right=False)
    
    # Conditionally one-hot encode
    if one_hot_encode_distance:
        df = pd.get_dummies(df, columns=distance_cols, drop_first=True)
    
    return df

df = process_and_encode_distance(df, one_hot_encode_distance=True)

In [6]:
df.head()


,url_id,Celková cena,Podlaží,Užitná plocha,Terasa,Výtah,Latitude,Longitude,Balkón,Plyn,...,Obchod_Far,Obchod_Very Far,Lékárna_Close,Lékárna_Medium Distance,Lékárna_Far,Lékárna_Very Far,Metro_Close,Metro_Medium Distance,Metro_Far,Metro_Very Far
0,3407975756,3681000.0,1.0,19.0,1,1,50.072829,14.459936,0,0,...,False,False,False,False,False,False,True,False,False,False
1,1385604428,11032000.0,3.0,71.0,0,1,50.072533,14.441122,1,1,...,False,False,False,False,False,False,False,False,False,False
2,983647308,6177000.0,3.0,55.0,0,1,50.098053,14.492726,0,1,...,False,False,True,False,False,False,True,False,False,False
3,1450153292,3425000.0,1.0,36.0,0,0,49.213194,16.602244,0,0,...,False,False,False,False,False,False,False,False,False,True
4,2906166604,11837000.0,5.0,64.0,0,1,50.094345,14.395542,1,0,...,False,False,True,False,False,False,False,False,False,False


In [7]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

missing_values

Počet vlastníků    17571
dtype: int64

In [9]:
df.to_csv("2023-10-09_data_cleaned.csv", index=False)